#ModLoop

ModLoop is a protocol for automated modeling of loops in protein structures. The server relies on the loop modeling routine in [MODELLER](https://salilab.org/modeller/) that predicts the loop conformations by satisfaction of spatial restraints, without relying on a database of known protein structures.

The ModLoop protocol can be run in Google Colab. The first step is to install the Modeller software by running the cell below:

In [ ]:
# Install Modeller from Sali lab website
modver = "10.8"
!wget "https://salilab.org/modeller/{modver}/modeller_{modver}-1_amd64.deb"
!apt install "./modeller_{modver}-1_amd64.deb"
!rm "modeller_{modver}-1_amd64.deb"
# Add Modeller Python modules to Colab's Python path
import sys
sys.path.append("/usr/lib/python3.9/dist-packages")

Next, configure the protocol by running the following cell and giving the [MODELLER license key](https://modbase.compbio.ucsf.edu/modloop/help#modkey), uploading the [starting structure](https://modbase.compbio.ucsf.edu/modloop/help#file) in PDB or mmCIF format, and specifying the [loops to refine](https://modbase.compbio.ucsf.edu/modloop/help#loop):

In [ ]:
# @title
import ipywidgets as widgets
from ipywidgets import GridspecLayout
grid = GridspecLayout(3, 2)
grid[0, 0] = widgets.Label("Modeller license key")
grid[0, 1] = key = widgets.Text()
grid[1, 0] = widgets.Label("Upload coordinate file")
grid[1, 1] = coord = widgets.FileUpload(multiple=False)
grid[2, 0] = widgets.Label("Enter loop segments")
grid[2, 1] = loops = widgets.Textarea()
grid

We use the input data to set up the protocol:

In [ ]:
# Add Modeller license key
with open(f"/usr/lib/modeller{modver}/modlib/modeller/config.py") as fh:
    inst_dir = fh.readline()
with open(f"/usr/lib/modeller{modver}/modlib/modeller/config.py", "w") as fh:
    fh.write(inst_dir)
    fh.write(f'license = {key.value!r}\n')

# Save uploaded file to local disk
in_fname = list(coord.value.keys())[0]
with open(in_fname, 'wb') as fh:
    fh.write(coord.value[in_fname]['content'])

def parse_loop_selection(loops):
    """Split out loop selection and check it"""
    import re
    # capitalize and remove spaces
    loops = re.sub(r'\s+', '', loops.upper())
    # replace null chain IDs with a single space
    loops = loops.replace("::", ": :")

    loop_data = loops.split(":")[:-1]

    # Make sure correct number of colons were given
    if len(loop_data) % 4 != 0:
        raise ValueError(
            "Syntax error in loop selection: check to make sure you "
            "have colons in the correct place (there should be a "
            "multiple of 4 colons)")

    total_res = 0
    start_res = []
    start_id = []
    end_res = []
    end_id = []
    loops = 0
    while loops*4+3 < len(loop_data) and loop_data[loops*4] != "":
        try:
            start_res.append(int(loop_data[loops*4]))
            end_res.append(int(loop_data[loops*4+2]))
        except ValueError:
            raise ValueError(
                "Residue indices are not numeric")
        start_id.append(loop_data[loops*4+1])
        end_id.append(loop_data[loops*4+3])
        # all the selected residues
        total_res += (end_res[-1] - start_res[-1] + 1)

        ################################
        # too long loops rejected
        if ((end_res[-1] - start_res[-1]) > 20
                or start_id[-1] != end_id[-1]
                or (end_res[-1] - start_res[-1]) < 0):
            raise ValueError(
                "The loop selected is too long (>20 residues) or "
                "shorter than 1 residue or not selected properly "
                "(syntax problem?) "
                "starting position %d:%s, ending position: %d:%s"
                % (start_res[-1], start_id[-1], end_res[-1], end_id[-1]))
        loops += 1

    ################################
    # too many or no residues rejected
    if total_res > 20:
        raise ValueError(
            "Too many loop residues have been selected "
            " (selected: %d > limit:20)!" % total_res)
    if total_res <= 0:
        raise ValueError(
            "No loop residues selected!")
    return loop_data

def get_output_header(loop_data, nmodel):
    """Return a suitable header for output model files"""
    residue_range = []
    for i in range(0, len(loop_data), 4):
        residue_range.append("   %s:%s-%s:%s" % tuple(loop_data[i:i + 4]))
    looplist = "\n".join(residue_range)
    return f"""
Dear User,

Coordinates for the lowest energy model (out of {nmodel} sampled)
are returned with the optimized loop regions, listed below:
{looplist}

for references please cite these two articles:

   A Fiser, RKG Do and A Sali,
   Modeling of loops in protein structures
   Prot. Sci. (2000) 9, 1753-1773

   A Fiser and A Sali,
   ModLoop: Automated modeling of loops in protein structures
   Bioinformatics. (2003) 18(19) 2500-01


For further inquiries, please contact: modloop@ucsf.edu

with best regards,
Andras Fiser


"""

def add_loop_header(model, loop_data, nmodel):
    """Add a header to the given model PDB or mmCIF file"""
    with open(model) as fin:
        contents = fin.read()
    prefix = '#' if model.endswith('.cif') else 'REMARK'
    with open(model, 'w') as fout:
        for line in get_output_header(loop_data, nmodel).split('\n'):
            if line == '':
                fout.write(prefix + '\n')
            else:
                fout.write(f'{prefix}     {line}\n')
        fout.write(contents)

# Check the provided set of loop residues to refine
loop_data = parse_loop_selection(loops.value)

The loop modeling protocol itself is just a short Python script that runs MODELLER on the input file uploaded earlier, selecting the loop residues given:

In [ ]:
from modeller import Environ, Selection, ModellerError
from modeller.automodel import LoopModel, refine
import sys

class MyLoop(LoopModel):
    def select_loop_atoms(self):
        rngs = []
        for i in range(0, len(loop_data), 4):
            rngs.append(self.residue_range("%s:%s" % tuple(loop_data[i:i+2]),
                                           "%s:%s" % tuple(loop_data[i+2:i+4])))
            if len(rngs[-1]) > 30:
                raise ModellerError("loop too long")
        s = Selection(rngs)
        if len(s.only_no_topology()) > 0:
            raise ModellerError("some selected residues have no topology")
        return s

def make_loop(taskid):
    logfile = f'{taskid}.log'
    print(f'Logging output to {logfile}')
    old_sys_stdout = sys.stdout
    try:
        sys.stdout = open(logfile, 'w')
        env = Environ(rand_seed=-1000-taskid)
        m = MyLoop(env, inimodel=in_fname, sequence='loop')
        if in_fname.endswith('.cif'):
            m.set_output_model_format('MMCIF')
        else:
            m.set_output_model_format('PDB')
        m.loop.md_level = refine.slow
        m.loop.starting_model = m.loop.ending_model = taskid
        m.make()
        return m.loop.outputs[0]
    finally:
        sys.stdout = old_sys_stdout

Finally, we can run the protocol in parallel. The exact same protocol is run 300 times on the same inputs but with a different random seed. (This will run faster if given a CPU with more cores.) The single structure with the lowest molecular PDF (molpdf) is then selected.

In [ ]:
import multiprocessing
import operator

nmodel = 300
with multiprocessing.Pool() as pool:
    best_model = min(pool.imap_unordered(make_loop, range(1, nmodel+1)),
                     key=operator.itemgetter('molpdf'))
print(f"Best model is {best_model['name']}")

# Add an informative ModLoop header to the best model PDB/mmCIF file
add_loop_header(best_model['name'], loop_data, nmodel)

Run the cell below to download the selected structure:

In [ ]:
from google.colab import files
files.download(best_model['name'])
